In [124]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import os
import sys
from PIL import Image
import pandas as pd
import numpy as np
import seaborn as sns
import scipy
import keras
import tensorflow as tf
import cv2
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.metrics import classification_report,confusion_matrix
from keras.models import Model as KerasModel
from keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU
from sklearn.utils import shuffle
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam

from tqdm import tqdm
np.random.seed(0)
tf.compat.v1.random.set_random_seed(0)

In [208]:
directory = os.path.join(os.getcwd(), 'data', 'dataset')
fake_directory = os.path.join(directory, 'fake')
real_directory = os.path.join(directory, 'real')

fake = []
real = []

In [126]:
for name in tqdm(os.listdir(fake_directory)):
  fake.append(os.path.join(fake_directory, name))

for name in tqdm(os.listdir(real_directory)):
  real.append(os.path.join(real_directory, name))

100%|██████████| 1000/1000 [00:00<00:00, 200052.66it/s]


In [200]:
y = np.ones(len(real))
x = np.zeros(len(fake))

label = np.concatenate([y, x])

names_list = real + fake

new_data, new_label = shuffle(names_list, label,  random_state = 0)

size = len(new_data)
train_size = int(size * 0.8) * -1
test_size = int(size * 0.2) * -1

train, test, y_train, y_test = new_data[:test_size], new_data[train_size:], new_label[:train_size], new_label[test_size:]

400


In [220]:
def decode_img(img):
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.uint8)
    img = tf.image.resize(img, [224, 224])
    return img

def get_label(file_path):
    cat = tf.strings.split(file_path, ':')[1]
    print(cat)
    if cat == b'fake':
        return 1
    return 0

def process_path(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [221]:
num_threads, num_epochs, train_len = 8, 20, len(train) 
train_ds = tf.data.Dataset.from_tensor_slices(train)
train_ds = train_ds.map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)

Tensor("args_0:0", shape=(), dtype=string)


In [206]:
train_ds = train_ds.shuffle(train_len)
train_ds = train_ds.batch(64)
train_ds = train_ds.prefetch(1)

In [203]:
train_ds

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [204]:
from tensorflow.keras.applications.vgg16 import VGG16

model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [180]:
opt = tf.keras.optimizers.Adam(
    learning_rate= 0.01,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False)

def loss_fn(labels, predictions):
    return tf.math.confusion_matrix(
    labels, predictions, num_classes=2, weights=None, dtype=tf.dtypes.int32,
    name=None).numpy()[0,1]

tp = tf.keras.metrics.TruePositives(thresholds=None, name=None, dtype=None)
tn = tf.keras.metrics.TrueNegatives(thresholds=None, name=None, dtype=None)
fp = tf.keras.metrics.FalsePositives(thresholds=None, name=None, dtype=None)
fn = tf.keras.metrics.FalseNegatives(thresholds=None, name=None, dtype=None)

In [181]:
classifier = Sequential()
classifier.add(model)
classifier.add(Flatten())
classifier.add(Dense(1024, activation='relu'))
classifier.add(Dense(1, activation = 'sigmoid'))
classifier.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy', tp,tn,fp,fn])

In [205]:
history = classifier.fit(train_ds, epochs = 20, shuffle=True, batch_size=32)

Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node 'strided_slice' defined at (most recent call last):
    File "c:\ProgramData\Anaconda3\envs\tf-venv\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\ProgramData\Anaconda3\envs\tf-venv\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\ProgramData\Anaconda3\envs\tf-venv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\ProgramData\Anaconda3\envs\tf-venv\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "c:\ProgramData\Anaconda3\envs\tf-venv\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\ProgramData\Anaconda3\envs\tf-venv\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\ProgramData\Anaconda3\envs\tf-ve

In [150]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(nb_epochs)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

NameError: name 'history' is not defined